## Intervista a Personaggi Storici Famosi

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel
from langchain_core.prompts import PromptTemplate
from typing import Dict, Any
import json
from langchain.chat_models import init_chat_model

In [2]:
def debug_print(data: Dict[str, Any]) -> Dict[str, Any]:
    print(f"\n\nDebug - Dati correnti: {json.dumps(data, indent=2)}\n\n") # indent=2
    return data

In [7]:
llm = init_chat_model("openai:gpt-4o-mini")

In [3]:
# Definizione dei prompt
interview_prompt = PromptTemplate.from_template(
   "Assumi il ruolo di {character}. Rispondi alla seguente domanda come se fossi davvero questo personaggio.\n\nDomanda: {question}"
)

verification_prompt = PromptTemplate.from_template(
    "{character} potrebbe rispondere nel modo seguente? {reply}\n\nAnalizza la risposta e spiega se è coerente con il personaggio e perché"
)

In [8]:
# Creazione della catena
chain = (
    # {"character":"...", "question":"..."}
    RunnableParallel(
        {
            "interview_response": interview_prompt | llm | StrOutputParser(),
            "character": RunnablePassthrough().pick("character"),
            "question": RunnablePassthrough().pick("question")
        }
    ) # {"interview_response":"...", "character":"...", "question":"..."}
    | RunnableLambda(debug_print)
    # {"interview_response":"...", "character":"...", "question":"..."}
    | RunnableLambda(lambda x: 
        {
            "character": x["character"],
            "reply": x["interview_response"],
            "original_question": x["question"]
        }) # {"reply":"...", "character":"...", "original_question":"..."}
    | RunnableParallel(
        {
            "interview_response": RunnablePassthrough().pick("reply"),
            "verification": verification_prompt | llm | StrOutputParser(),
            "original_question": RunnablePassthrough().pick("original_question")
        }
    ) # {"interview_response":"...", "verification":"...", "original_question":"..."}
)

In [9]:
# Uso della catena
result = chain.invoke({
    "character": "Albert Einstein",
    "question": "Qual è la sua opinione sulla relatività dello spazio-tempo?"
})

print("\n=============\n")
print("Domanda originale:", result["original_question"])
print("\n---\n")
print("\nRisposta del personaggio:", result["interview_response"])
print("\n---\n")
print("\nVerifica della risposta:", result["verification"])



Debug - Dati correnti: {
  "interview_response": "La relativit\u00e0 dello spazio-tempo rappresenta una delle scoperte pi\u00f9 rivoluzionarie nella comprensione del nostro universo. Quando parlo di relativit\u00e0, mi riferisco a due teorie fondamentali: la relativit\u00e0 ristretta, formulata nel 1905, e la relativit\u00e0 generale, pubblicata nel 1915.\n\nNella relativit\u00e0 ristretta, ho dimostrato che le leggi della fisica sono le stesse per tutti gli osservatori in moto uniforme e che la velocit\u00e0 della luce nel vuoto \u00e8 costante, indipendentemente dal movimento dell'osservatore. Questo ha portato a risultati sorprendenti, come la dilatazione del tempo e la contrazione delle lunghezze, che sfidano la nostra intuizione quotidiana.\n\nLa relativit\u00e0 generale, d'altro canto, ci offre una nuova visione della gravit\u00e0. Non pi\u00f9 vista come una forza misteriosa a distanza, ma come una curvatura dello spazio-tempo causata dalla presenza di massa. Gli oggetti si mu